## 依赖安装

In [ ]:
!pip install h5py
!pip install Pyaml

## 参数处理

In [91]:
%cd fgcnn/fuxi_pipline

import sys
sys.path.append('../')
import os
import logging
from datetime import datetime
from fuxictr import datasets
from fuxictr.datasets.taobao import FeatureEncoder
from fuxictr.features import FeatureMap
from fuxictr.utils import load_config, set_logger, print_to_json
from fuxictr.pytorch.models import FGCNN
from fuxictr.pytorch.torch_utils import seed_everything
import torch

feature_cols = [{'name': ["userid","adgroup_id","pid","cate_id","campaign_id","customer","brand","cms_segid",
                            "cms_group_id","final_gender_code","age_level","pvalue_level","shopping_level","occupation"],
                    'active': True, 'dtype': 'str', 'type': 'categorical'}]
label_col = {'name': 'clk', 'dtype': float}

params = {'model_id': 'FGCNN_demo',
            'dataset_id': 'taobao_tiny',
            'train_data': '../data/tiny_data/train_sample.csv',
            'valid_data': '../data/tiny_data/valid_sample.csv',
            'test_data': '../data/tiny_data/test_sample.csv',
            'model_root': '../checkpoints/',
            'data_root': '../data/',
            'feature_cols': feature_cols,
            'label_col': label_col,
            'embedding_regularizer': 0,
            'net_regularizer': 0,
            'dnn_hidden_units': [128, 64],
            "channels": [14, 16, 18, 20],
            "kernel_heights": [7, 7, 7, 7],
            'hidden_activations': "relu",
            'learning_rate': 1e-3,
            'net_dropout': 0,
            'batch_norm': False,
            'optimizer': 'adam',
            'task': 'binary_classification',
            'loss': 'binary_crossentropy',
            'metrics': ['logloss', 'AUC'],
            'min_categr_count': 1,
            'embedding_dim': 10,
            "pooling_sizes": [2, 2, 2, 2],
            "recombined_channels": [3, 3, 3, 3],
            'batch_size': 1,
            'epochs': 3,
            'shuffle': True,
            'seed': 2019,
            'monitor': 'AUC',
            'monitor_mode': 'max',
            'use_hdf5': True,
            'pickle_feature_encoder': True,
            'save_best_only': True,
            'every_x_epochs': 1,
            'patience': 2,
            'num_workers': 1,
            'data_block_size': -1,
            'verbose': 1,
            'version': 'pytorch',
            'gpu': -1}

set_logger(params)
logging.info(print_to_json(params))
seed_everything(seed=params['seed'])

# Set feature_encoder that defines how to preprocess data
feature_encoder = FeatureEncoder(feature_cols, 
                                    label_col, 
                                    dataset_id=params['dataset_id'], 
                                    data_root=params["data_root"])
# Build dataset from csv to h5
datasets.build_dataset(feature_encoder, 
                        train_data=params["train_data"], 
                        valid_data=params["valid_data"], 
                        test_data=params["test_data"])
# Get feature_map that defines feature specs
feature_map = feature_encoder.feature_map
# Get train and validation data generator from h5
data_dir = os.path.join(params['data_root'], params['dataset_id'])
train_gen, valid_gen = datasets.h5_generator(feature_map, 
                                                stage='train', 
                                                train_data=os.path.join(data_dir, 'train.h5'),
                                                valid_data=os.path.join(data_dir, 'valid.h5'),
                                                batch_size=params['batch_size'],
                                                shuffle=params['shuffle'])                                      
torch_model = FGCNN(feature_map, **params)

2022-05-23 17:56:47,773 P75318 INFO {
    "batch_norm": "False",
    "batch_size": "1",
    "channels": "[14, 16, 18, 20]",
    "data_block_size": "-1",
    "data_root": "../data/",
    "dataset_id": "taobao_tiny",
    "dnn_hidden_units": "[128, 64]",
    "embedding_dim": "10",
    "embedding_regularizer": "0",
    "epochs": "3",
    "every_x_epochs": "1",
    "feature_cols": "[{'name': ['userid', 'adgroup_id', 'pid', 'cate_id', 'campaign_id', 'customer', 'brand', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level', 'pvalue_level', 'shopping_level', 'occupation'], 'active': True, 'dtype': 'str', 'type': 'categorical'}]",
    "gpu": "-1",
    "hidden_activations": "relu",
    "kernel_heights": "[7, 7, 7, 7]",
    "label_col": "{'name': 'clk', 'dtype': <class 'float'>}",
    "learning_rate": "0.001",
    "loss": "binary_crossentropy",
    "metrics": "['logloss', 'AUC']",
    "min_categr_count": "1",
    "model_id": "FGCNN_demo",
    "model_root": "../checkpoints/",
    "monitor

/hy-tmp/Paddle-fgcnn-version1/fgcnn/fuxi_pipline


2022-05-23 17:56:48,014 P75318 INFO Preprocess feature columns...
2022-05-23 17:56:48,024 P75318 INFO Transform feature columns...
2022-05-23 17:56:48,027 P75318 INFO Saving data to h5: ../data/taobao_tiny/valid.h5
2022-05-23 17:56:48,141 P75318 INFO Preprocess feature columns...
2022-05-23 17:56:48,148 P75318 INFO Transform feature columns...
2022-05-23 17:56:48,151 P75318 INFO Saving data to h5: ../data/taobao_tiny/test.h5
2022-05-23 17:56:48,268 P75318 INFO Transform csv data to h5 done.
2022-05-23 17:56:48,269 P75318 INFO Loading data...
2022-05-23 17:56:48,270 P75318 INFO Loading data from h5: ../data/taobao_tiny/train.h5
2022-05-23 17:56:48,272 P75318 INFO Loading data from h5: ../data/taobao_tiny/valid.h5
2022-05-23 17:56:48,274 P75318 INFO Train samples: total/100, pos/4, neg/96, ratio/4.00%, blocks/1
2022-05-23 17:56:48,275 P75318 INFO Validation samples: total/100, pos/4, neg/96, ratio/4.00%, blocks/1
2022-05-23 17:56:48,276 P75318 INFO Loading train data done.


## FuxiCTR-FGCNN

In [165]:
%cd fgcnn/fuxi_pipline
import numpy as np
from torch import nn
import torch
from fuxictr.pytorch.models.base_model import BaseModel
from fuxictr.pytorch.layers import EmbeddingLayer#, InnerProductLayer
from fuxictr.pytorch.torch_utils import set_activation

class fuxi_FGCNN(BaseModel):
    def __init__(self, 
                 feature_map, 
                 model_id="FGCNN", 
                 gpu=-1, 
                 task="binary_classification", 
                 learning_rate=1e-3, 
                 embedding_dim=10, 
                 share_embedding=False,
                 channels=[14, 16, 18, 20],
                 kernel_heights=[7, 7, 7, 7],
                 pooling_sizes=[2, 2, 2, 2],
                 recombined_channels=[2, 2, 2, 2],
                 conv_activation="Tanh",
                 conv_batch_norm=True,
                 dnn_hidden_units=[128, 64],
                 dnn_activations="ReLU",
                 dnn_batch_norm=False, 
                 embedding_regularizer=None, 
                 net_regularizer=None,
                 net_dropout=0,
                 **kwargs):
        super(fuxi_FGCNN, self).__init__(feature_map, 
                                    model_id=model_id, 
                                    gpu=gpu, 
                                    embedding_regularizer=embedding_regularizer, 
                                    net_regularizer=net_regularizer,
                                    **kwargs)
        self.share_embedding = share_embedding
        self.embedding_layer = EmbeddingLayer(feature_map, embedding_dim)
        if not self.share_embedding:
            self.fg_embedding_layer = EmbeddingLayer(feature_map, embedding_dim)
        num_fields = feature_map.num_fields
        channels, kernel_heights, pooling_sizes, recombined_channels \
            = self.validate_input(channels, 
                                  kernel_heights, 
                                  pooling_sizes, 
                                  recombined_channels)
        self.fgcnn_layer = FGCNN_Layer(num_fields, 
                                       embedding_dim,
                                       channels=channels, 
                                       kernel_heights=kernel_heights, 
                                       pooling_sizes=pooling_sizes,
                                       recombined_channels=recombined_channels,
                                       activation=conv_activation,
                                       batch_norm=conv_batch_norm)
        input_dim, total_features = self.compute_input_dim(embedding_dim, 
                                                           num_fields, 
                                                           channels, 
                                                           pooling_sizes, 
                                                           recombined_channels)
        self.inner_product_layer = InnerProductLayer(total_features)#, output="inner_product")
        self.dnn = MLP_Layer(input_dim=input_dim,
                             output_dim=1, 
                             hidden_units=dnn_hidden_units,
                             hidden_activations=dnn_activations,
                             final_activation=nn.Sigmoid(),
                             dropout_rates=net_dropout,
                             batch_norm=dnn_batch_norm)

    def compute_input_dim(self, 
                          embedding_dim, 
                          num_fields, 
                          channels, 
                          pooling_sizes, 
                          recombined_channels):
        total_features = num_fields
        input_height = num_fields
        for i in range(len(channels)):
            input_height = int(np.ceil(input_height / pooling_sizes[i]))
            total_features += input_height * recombined_channels[i]
        input_dim = int(total_features * (total_features - 1) / 2) \
                  + total_features * embedding_dim
        return input_dim, total_features

    def validate_input(self, 
                       channels, 
                       kernel_heights, 
                       pooling_sizes, 
                       recombined_channels):
        if not isinstance(kernel_heights, list):
            kernel_heights = [kernel_heights] * len(channels)
        if not isinstance(pooling_sizes, list):
            pooling_sizes = [pooling_sizes] * len(channels)
        if not isinstance(recombined_channels, list):
            recombined_channels = [recombined_channels] * len(channels)
        if not (len(channels) == len(kernel_heights) == len(pooling_sizes) == len(recombined_channels)):
            raise ValueError("channels, kernel_heights, pooling_sizes, and recombined_channels \
                              should have the same length.")
        return channels, kernel_heights, pooling_sizes, recombined_channels

    def forward(self, inputs):
        """
        Inputs: [X, y]
        """
        X, y = self.inputs_to_device(inputs)
        feature_emb = self.embedding_layer(X)
        if not self.share_embedding:
            feature_emb2 = self.fg_embedding_layer(X)
        else:
            feature_emb2 = feature_emb
        conv_in = torch.unsqueeze(feature_emb2, 1)
        new_feature_emb = self.fgcnn_layer(conv_in)
        combined_feature_emb = torch.cat([feature_emb, new_feature_emb], dim=1)
        inner_product_vec = self.inner_product_layer(combined_feature_emb)
        dense_input = torch.cat([combined_feature_emb.flatten(start_dim=1), inner_product_vec], dim=1)
        y_pred = self.dnn(dense_input)
        return y_pred
        return_dict = {"y_true": y, "y_pred": y_pred}
        return return_dict


class FGCNN_Layer(nn.Module):
    """
    Input X: tensor of shape (batch_size, 1, num_fields, embedding_dim)
    """
    def __init__(self, 
                 num_fields, 
                 embedding_dim,
                 channels=[3], 
                 kernel_heights=[3], 
                 pooling_sizes=[2],
                 recombined_channels=[2],
                 activation="Tanh",
                 batch_norm=False):
        super(FGCNN_Layer, self).__init__()
        self.embedding_dim = embedding_dim
        conv_list = []
        recombine_list = []
        self.channels = [1] + channels # input channel = 1
        input_height = num_fields
        for i in range(1, len(self.channels)):
            in_channel = self.channels[i - 1]
            out_channel = self.channels[i]
            kernel_height = kernel_heights[i - 1]
            pooling_size = pooling_sizes[i - 1]
            recombined_channel = recombined_channels[i - 1]
            conv_list.append(nn.Sequential(
                nn.Conv2d(in_channel, out_channel, 
                        kernel_size=(kernel_height, 1), 
                        padding=(int((kernel_height - 1) / 2), 0)),
                nn.BatchNorm2d(out_channel),
                nn.Tanh(),
                nn.MaxPool2d((pooling_size, 1), stride=(pooling_size, 1), padding=(input_height % pooling_size, 0))
            ))
            input_height = int(np.ceil(input_height / pooling_size))
            input_dim =  input_height * embedding_dim * out_channel
            output_dim = input_height * embedding_dim * recombined_channel
            recombine_layer = nn.Sequential(nn.Linear(input_dim, output_dim),
                                            nn.Tanh())
            recombine_list.append(recombine_layer)
        self.conv_layers = nn.ModuleList(conv_list)
        self.recombine_layers = nn.ModuleList(recombine_list)

    def forward(self, X):
        conv_out = X
        new_feature_list = []
        for i in range(len(self.channels) - 1):
            conv_out = self.conv_layers[i](conv_out)
            flatten_out = torch.flatten(conv_out, start_dim=1)
            recombine_out = self.recombine_layers[i](flatten_out)
            new_feature_list.append(recombine_out.reshape(X.size(0), -1, self.embedding_dim))
        new_feature_emb = torch.cat(new_feature_list, dim=1)
        return new_feature_emb

class MLP_Layer(nn.Module):
    def __init__(self, 
                 input_dim, 
                 output_dim=None, 
                 hidden_units=[], 
                 hidden_activations="ReLU",
                 final_activation=None, 
                 dropout_rates=[], 
                 batch_norm=False, 
                 use_bias=True):
        super(MLP_Layer, self).__init__()
        dense_layers = []
        if not isinstance(dropout_rates, list):
            dropout_rates = [dropout_rates] * len(hidden_units)
        if not isinstance(hidden_activations, list):
            hidden_activations = [hidden_activations] * len(hidden_units)
        hidden_activations = [set_activation(x) for x in hidden_activations]
        hidden_units = [input_dim] + hidden_units
        for idx in range(len(hidden_units) - 1):
            dense_layers.append(nn.Linear(hidden_units[idx], hidden_units[idx + 1], bias=use_bias))
            if batch_norm:
                dense_layers.append(nn.BatchNorm1d(hidden_units[idx + 1]))
            if hidden_activations[idx]:
                dense_layers.append(hidden_activations[idx])
            if dropout_rates[idx] > 0:
                dense_layers.append(nn.Dropout(p=dropout_rates[idx]))
        if output_dim is not None:
            dense_layers.append(nn.Linear(hidden_units[-1], output_dim, bias=use_bias))
        if final_activation is not None:
            dense_layers.append(set_activation(final_activation))
        self.dnn = nn.Sequential(*dense_layers) # * used to unpack list
    
    def forward(self, inputs):
        return self.dnn(inputs)

class InnerProductLayer(nn.Module):
    def __init__(self, num_fields=None):
        super(InnerProductLayer, self).__init__()
        self.interaction_units = int(num_fields * (num_fields - 1) / 2)
        self.upper_triange_mask = torch.triu(torch.ones(num_fields, num_fields), 1).type(torch.BoolTensor)

    def forward(self, feature_emb):
        inner_product_matrix = torch.bmm(feature_emb, feature_emb.transpose(1, 2))
        flat_upper_triange = torch.masked_select(inner_product_matrix, self.upper_triange_mask)
        return flat_upper_triange.view(-1, self.interaction_units)

torch_model = fuxi_FGCNN(feature_map, **params)

/hy-tmp/Paddle-fgcnn-version1/fgcnn/fuxi_pipline


In [166]:
# pipeline
import torch
import pickle
import paddle
# from paddle_net import FGCNN as paddle_FGCNN
from reprod_log import ReprodLogger, ReprodDiffHelper
# path
%cd /hy-tmp/Paddle-fgcnn-version1/fgcnn/fuxi_pipline
# data
pipline_data = pickle.load(open('data/sample.pkl', 'rb'))
# torch
torch_para = torch.load('data/torch_para.pt')
del torch_para["inner_product_layer.field_p"]
del torch_para["inner_product_layer.field_q"]
del torch_para["inner_product_layer.upper_triange_mask"]
torch_model.load_state_dict(torch_para)

/hy-tmp/Paddle-fgcnn-version1/fgcnn/fuxi_pipline


<All keys matched successfully>

In [158]:
#step1
torch_model.eval()
torch_forward = []
for data in pipline_data:
    data, label = data[:-1], data[-1]
    # torch
    torch_inputs = list(map(torch.tensor, [data.reshape((1,-1)), label]))
    torch_out = torch_model.forward(torch_inputs)
    torch_forward.append(torch_out.detach().numpy())

# torch
reprod_logger = ReprodLogger()
reprod_logger.add("logits", np.array(torch_forward))
reprod_logger.save("Step1/forward_torch.npy")

In [161]:
#step2
torch_model.eval()
torch_cost = []
for data in pipline_data:
    data, label = data[:-1], data[-1]
    # torch
    torch_inputs = list(map(torch.tensor, [data.reshape((1,-1)), label]))
    torch_out = torch_model.forward(torch_inputs)#['y_pred']
    torch_cost.append(torch.nn.functional.binary_cross_entropy(torch_out, torch.Tensor([label]).reshape((1,1))).detach().numpy())
# torch
reprod_logger = ReprodLogger()
reprod_logger.add("binary_cross_entropy", np.array(torch_cost))
reprod_logger.save("Step2/metric_torch.npy")

In [164]:
#step4
# torch
torch_model.load_state_dict(torch_para)
torch_model.eval()
torch_optim = torch.optim.Adam(params=torch_model.parameters())
# paddle
paddle_model.load_dict(paddle_para)
paddle_model.eval()
paddle_optim = paddle.optimizer.Adam(parameters=paddle_model.parameters())
# grad
def save_paddle_hook_fn(grad):
    global paddle_grad
    paddle_grad.append(grad)

def save_torch_hook_fn(grad):
    global torch_grad
    torch_grad.append(grad)
t_out = []
torch_loss = []
labels = []
torch_grad= []
for data in pipline_data[:2]:
    data, label = data[:-1], data[-1]
    # torch
    torch_optim.zero_grad()
    torch_inputs = list(map(torch.tensor, [data.reshape((1,-1)), label]))
    torch_out = torch_model.forward(torch_inputs)#['y_pred']
    t_loss = torch.nn.functional.binary_cross_entropy(torch_out, torch.Tensor([label]).reshape((1,1)))
    # t_loss.register_hook(lambda grad: print('t:', grad.detach().numpy()))
    t_loss.backward()
    # for name, tensor in torch_model.named_parameters():
    #     grad = tensor.grad.numpy()
    #     torch_grad.append((name, grad))
    torch_optim.step()
    # test
    t_out.append(torch_out.detach().numpy())
    torch_loss.append(t_loss.detach().numpy())
    labels.append(label)
# torch
reprod_logger = ReprodLogger()
reprod_logger.add("loss", np.array(torch_loss))
reprod_logger.save("Step4/bp_align_torch.npy")

In [167]:
# step 5
# torch
torch_model.load_state_dict(torch_para)
torch_model.eval()
torch_optim = torch.optim.Adam(
            params=torch_model.parameters(),
            lr=0.001)

t_out = []
torch_loss = []
for epoch in range(2):
    for data in pipline_data:
        data, label = data[:-1], data[-1]
        torch_optim.zero_grad()
        torch_inputs = list(map(torch.tensor, [data.reshape((1,-1)), label]))
        torch_out = torch_model.forward(torch_inputs)#['y_pred']
        t_loss = torch.nn.functional.binary_cross_entropy(torch_out, torch.Tensor([label]).reshape((1,1)))
        # t_loss.register_hook(lambda grad: print('t:', grad.detach().numpy()))
        t_loss.backward()
        torch_optim.step()
        # test
        t_out.append(torch_out.detach().numpy())
        torch_loss.append(t_loss.detach().numpy())
    # print(torch_loss)
    # print(paddle_loss)
# torch
reprod_logger = ReprodLogger()
reprod_logger.add("train", np.array(torch_loss))
reprod_logger.save("Step5/train_torch1.npy")

## paddle-FGCNN

In [182]:
import paddle
import paddle.nn as nn
import numpy as np
from paddle.nn import functional as F


class paddle_FGCNN(nn.Layer):
    def __init__(self, sparse_num_field, sparse_feature_size,
                 sparse_feature_name, sparse_feature_dim, conv_kernel_width,
                 conv_filters, new_maps, pooling_width,
                 dnn_hidden_units, dnn_dropout):
        '''
        Parameters
            sparse_num_field - 
            sparse_feature_size - 
            sparse_feature_name - 
            sparse_feature_dim - 
            conv_kernel_width - 
            conv_filters - 
            new_maps - 
            pooling_width - 
            dnn_hidden_units - 
            dnn_dropout - 
        '''
        super(paddle_FGCNN, self).__init__()
        self.sparse_num_field = sparse_num_field
        self.sparse_feature_size = sparse_feature_size
        self.sparse_feature_name = sparse_feature_name
        self.sparse_feature_dim = sparse_feature_dim
        self.conv_filters = conv_filters
        self.conv_kernel_width = conv_kernel_width
        self.new_maps = new_maps
        self.pooling_width = pooling_width
 
        
        self.fg_embedding = nn.LayerList([
            EmbeddingLayer(
                num_embeddings=self.sparse_feature_size[i],
                embedding_dim=self.sparse_feature_dim,
                feature_name=self.sparse_feature_name[i] + '_fg_emd'
            ) for i in range(self.sparse_num_field)])

        self.embedding = nn.LayerList([
            EmbeddingLayer(
                num_embeddings=self.sparse_feature_size[i],
                embedding_dim=self.sparse_feature_dim,
                feature_name=self.sparse_feature_name[i] + '_emd'
            ) for i in range(self.sparse_num_field)])

        self.fgcnn = FGCNNLayer(self.sparse_num_field, self.sparse_feature_dim,
                                self.conv_filters, self.conv_kernel_width, 
                                self.new_maps, self.pooling_width)
        input_dim, total_features = self.compute_input_dim(
            self.sparse_feature_dim,
            self.sparse_num_field,
            self.conv_filters,
            self.pooling_width, 
            self.new_maps)
        self.inner_product_layer = InnerProductLayer(total_features)
        self.dnn = DNNLayer(input_dim, dnn_hidden_units, dnn_dropout)

    def compute_input_dim(self, embedding_dim, num_fields, channels, 
                          pooling_sizes, recombined_channels):
        total_features = num_fields
        input_height = num_fields
        for i in range(len(channels)):
            input_height = int(np.ceil(input_height / pooling_sizes[i]))
            total_features += input_height * recombined_channels[i]
        input_dim = int(total_features * (total_features - 1) / 2) \
                  + total_features * embedding_dim
        return input_dim, total_features

    def forward(self, inputs):
        inputs = paddle.to_tensor(inputs).reshape((-1, self.sparse_num_field))
        fg_input_list = []
        origin_input_list = []
        for i in range(self.sparse_num_field):
            fg_input_list.append(
                self.fg_embedding[i](inputs[:, i].astype('int64'))
                .reshape((-1, 1, self.sparse_feature_dim)))
            origin_input_list.append(
                self.embedding[i](inputs[:, i].astype('int64'))
                .reshape((-1, 1, self.sparse_feature_dim)))
        fg_input = paddle.concat(fg_input_list, axis=1)
        origin_input = paddle.concat(origin_input_list, axis=1)
        new_features = self.fgcnn(fg_input)
        combined_input = paddle.concat([origin_input, new_features], axis=1)
        linear_signal = paddle.flatten(combined_input, start_axis=1)
        inner_product_vec = self.inner_product_layer(combined_input)
        # print('linear_signal:', linear_signal.shape)
        # print('inner_prodcut', inner_product.shape)
        dnn_input = paddle.concat([linear_signal, inner_product_vec], axis=1)
        # dnn_input = linear_signal
        # return dnn_input
        # print('dnn_input:', dnn_input.shape)
        y_pred = self.dnn(dnn_input)
        return y_pred

class EmbeddingLayer(nn.Layer):
    def __init__(self, num_embeddings, embedding_dim, feature_name):
        super(EmbeddingLayer, self).__init__()
        self.embedding = nn.Embedding(
            num_embeddings=num_embeddings,
            embedding_dim=embedding_dim,
            name=feature_name,
            sparse=True
        )

    def forward(self, inputs):
        return self.embedding(inputs)
    
class FGCNNLayer(nn.Layer):
    def __init__(self, sparse_num_field, embedding_size, filters, kernel_width, new_maps, pooling_width):
        super(FGCNNLayer, self).__init__()
        self.embedding_size = embedding_size
        conv_list = []
        recombine_list = []
        self.filters = [1] + filters # input channel = 1
        input_height = sparse_num_field
        for i in range(1, len(self.filters)):
            in_channel = self.filters[i - 1]
            out_channel = self.filters[i]
            kernel_height = kernel_width[i - 1]
            pooling_size = pooling_width[i - 1]
            recombined_channel = new_maps[i - 1]
            conv_layer = nn.Sequential(
                nn.Conv2D(
                    in_channels=in_channel, 
                    out_channels=out_channel, 
                    kernel_size=(kernel_height, 1), 
                    padding=(int((kernel_height - 1) / 2), 0)),
                nn.BatchNorm2D(out_channel),
                nn.Tanh(),
                nn.MaxPool2D(
                    kernel_size=(pooling_size, 1), 
                    stride=(pooling_size, 1),
                    padding=(input_height % pooling_size, 0)))
            # )
            conv_list.append(conv_layer)

            input_height = int(np.ceil(input_height / pooling_size))
            input_dim =  input_height * embedding_size * out_channel
            output_dim = input_height * embedding_size * recombined_channel
            # print('in: %d, out:%d' % (input_dim, output_dim))
            recombine_layer = nn.Sequential(
                nn.Linear(
                    in_features=input_dim, 
                    out_features=output_dim), 
                nn.Tanh())
            recombine_list.append(recombine_layer)

        self.conv_layers = nn.LayerList(conv_list)
        self.recombine_layers = nn.LayerList(recombine_list)

    def forward(self, inputs):
        conv_out = inputs.unsqueeze(1)
        new_feature_list = []
        for i in range(len(self.filters) - 1):
            conv_out = self.conv_layers[i](conv_out)
            flatten_out = paddle.flatten(conv_out, start_axis=1)
            # return flatten_out
            recombine_out = self.recombine_layers[i](flatten_out)
            # print(flatten_out.shape)
            # return recombine_out
            recombine_out = recombine_out.reshape((inputs.shape[0], -1, self.embedding_size))
            new_feature_list.append(recombine_out)
        # return new_feature_list
        new_features = paddle.concat(new_feature_list, axis=1)
        return new_features

class InnerProductLayer(nn.Layer):
    """ output: product_sum_pooling (bs x 1), 
                Bi_interaction_pooling (bs * dim), 
                inner_product (bs x f2/2), 
                elementwise_product (bs x f2/2 x emb_dim)
    """
    def __init__(self, num_fields=None):
        super(InnerProductLayer, self).__init__()
        if num_fields is None:
            raise ValueError("num_fields is required")
        else:
            self.num_fields = num_fields
            self.interaction_units = int(num_fields * (num_fields - 1) / 2)
            
    def forward(self, feature_emb):
        onemask = paddle.ones(shape=[feature_emb.shape[0],self.num_fields, self.num_fields],dtype='int32')
        tri = paddle.triu(onemask,1)
        upper_triange_mask = paddle.cast(tri,'bool')
        inner_product_matrix = paddle.bmm(feature_emb, paddle.transpose(feature_emb, perm=[0, 2, 1]))
        flat_upper_triange = paddle.masked_select(inner_product_matrix, upper_triange_mask)
        return flat_upper_triange.reshape([-1, self.interaction_units])


class DNNLayer(nn.Layer):
    def __init__(self, inputs_dim, hidden_units, dropout_rate):
        super(DNNLayer, self).__init__()
        self.dropout_rate = dropout_rate
        self.dropout = nn.Dropout(dropout_rate)

        hidden_units = [inputs_dim] + list(hidden_units)
        layers = []
        for i in range(len(hidden_units) - 1):
            layers.append(nn.Linear(
                in_features=hidden_units[i], 
                out_features=hidden_units[i + 1],
                name='dnn_%d' % i))
            layers.append(nn.ReLU())
            if self.dropout_rate > 0:
                layers.append(self.dropout)
        layers.append(nn.Linear(
            in_features=hidden_units[-1],
            out_features=1,
            name='dnn_%d' % (i+1)))
        layers.append(nn.Sigmoid())
        self.dnn = nn.Sequential(*layers)
        
    def forward(self, inputs):
        return self.dnn(inputs)

# hyperparameters
sparse_num_field = 14
sparse_feature_size = [25, 95, 3, 48, 98, 97, 66, 10, 10, 3, 6, 3, 4, 3]
sparse_feature_name = ['userid', 'adgroup_id', 'pid', 'cate_id', 'campaign_id', 'customer', 'brand', 'cms_segid', 'cms_group_id', 'final_gender_code', 'age_level', 'pvalue_level', 'shopping_level', 'occupation']
sparse_feature_dim = 10
conv_kernel_width =  [7, 7, 7, 7]
conv_filters =  [14, 16, 18, 20]
new_maps =  [3, 3, 3, 3]
pooling_width =  [2, 2, 2, 2]
dnn_hidden_units =  [128, 64]
dnn_dropout =  0.0
paddle_model = paddle_FGCNN(sparse_num_field, sparse_feature_size,
                 sparse_feature_name, sparse_feature_dim, conv_kernel_width,
                 conv_filters, new_maps, pooling_width,
                 dnn_hidden_units, dnn_dropout)

In [193]:
# pipeline
import torch
import pickle
import paddle
# from paddle_net import FGCNN as paddle_FGCNN
from reprod_log import ReprodLogger, ReprodDiffHelper
# data
pipline_data = pickle.load(open('data/sample.pkl', 'rb'))
# torch
torch_para = torch.load('data/torch_para.pt')
del torch_para["inner_product_layer.field_p"]
del torch_para["inner_product_layer.field_q"]
del torch_para["inner_product_layer.upper_triange_mask"]
# name map
name_map = {k:v for k,v in enumerate(sparse_feature_name)}
# fuxi
torch_linear_key = \
    ['dnn.dnn.%d.weight' % i for i in [0, 2, 4]] + \
    ['dnn.dnn.%d.bias' % i for i in [0, 2, 4]] + \
    ['fgcnn_layer.recombine_layers.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn_layer.recombine_layers.%d.0.bias' % i for i in range(4)] + \
    ['fgcnn_layer.conv_layers.%d.1.running_mean' % i for i in range(4)] + \
    ['fgcnn_layer.conv_layers.%d.1.running_var' % i for i in range(4)] 
torch_other_key = \
    ['embedding_layer.embedding_layer.embedding_layer.%s.weight' % v for v in name_map.values()] + \
    ['fg_embedding_layer.embedding_layer.embedding_layer.%s.weight' % v for v in name_map.values()] + \
    ['fgcnn_layer.conv_layers.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn_layer.conv_layers.%d.1.weight' % i for i in range(4)] + \
    ['fgcnn_layer.conv_layers.%d.0.bias' % i for i in range(4)] + \
    ['fgcnn_layer.conv_layers.%d.1.bias' % i for i in range(4)]
# torch_inner_key = 'inner_product_layer.upper_triange_mask'
# paddle
paddle_linear_key = \
    ['dnn.dnn.%d.weight' % i for i in [0, 2, 4]] + \
    ['dnn.dnn.%d.bias' % i for i in [0, 2, 4]] + \
    ['fgcnn.recombine_layers.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn.recombine_layers.%d.0.bias' % i for i in range(4)] + \
    ['fgcnn.conv_layers.%d.1._mean' % i for i in range(4)] + \
    ['fgcnn.conv_layers.%d.1._variance' % i for i in range(4)]
paddle_other_key = \
    ['embedding.%d.embedding.weight' % k for k in name_map.keys()] + \
    ['fg_embedding.%d.embedding.weight' % k for k in name_map.keys()] + \
    ['fgcnn.conv_layers.%d.0.weight' % i for i in range(4)] + \
    ['fgcnn.conv_layers.%d.1.weight' % i for i in range(4)] + \
    ['fgcnn.conv_layers.%d.0.bias' % i for i in range(4)] + \
    ['fgcnn.conv_layers.%d.1.bias' % i for i in range(4)]
# paddle_inner_key = 'inner_product_layer.upper_triange_mask'
# fuxi2paddle
key_map = {key_t:key_p for key_t, key_p in zip(torch_other_key, paddle_other_key)}
paddle_para = {key_map[k]:paddle.to_tensor(v.numpy()) for k, v in torch_para.items() if k in torch_other_key}
for t, p in zip(torch_linear_key, paddle_linear_key):
    if t in torch_para.keys():
        paddle_para[p] = paddle.to_tensor(torch_para[t].T.numpy())
paddle_model.load_dict(paddle_para)

In [ ]:
# step1
paddle_model.eval()
paddle_forward = []
for data in pipline_data:
    data, label = data[:-1], data[-1]
    paddle_out = paddle_model.forward(data)
    paddle_forward.append(paddle_out.numpy())

# paddle
reprod_logger = ReprodLogger()
reprod_logger.add("logits", np.array(paddle_forward))
reprod_logger.save("Step1/forward_paddle.npy")
# diff
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("Step1/forward_torch.npy")
paddle_info = diff_helper.load_info("Step1/forward_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="Diff/forward_diff.log")

In [ ]:
# step2
paddle_model.eval()
paddle_cost = []
for data in pipline_data:
    data, label = data[:-1], data[-1]
    # paddle
    paddle_out = paddle_model.forward(data)
    paddle_cost.append(paddle.nn.functional.binary_cross_entropy(paddle_out, paddle.to_tensor(float(label)).reshape((1,1))).numpy()[0])
# paddle
reprod_logger = ReprodLogger()
reprod_logger.add("binary_cross_entropy", np.array(paddle_cost))
reprod_logger.save("Step2/metric_paddle.npy")
# diff
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("Step2/metric_torch.npy")
paddle_info = diff_helper.load_info("Step2/metric_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="Diff/metric_diff.log")

In [ ]:
#step 4
# paddle
paddle_model.load_dict(paddle_para)
paddle_model.eval()
paddle_optim = paddle.optimizer.Adam(parameters=paddle_model.parameters())
# grad
def save_paddle_hook_fn(grad):
    global paddle_grad
    paddle_grad.append(grad)

p_out = []
paddle_loss = []
labels = []
paddle_grad = []
for data in pipline_data[:2]:
    data, label = data[:-1], data[-1]
    # paddle
    paddle_optim.clear_grad()
    paddle_out = paddle_model.forward(data)
    print(paddle_out)
    p_loss = paddle.nn.functional.binary_cross_entropy(paddle_out, paddle.to_tensor(float(label)).reshape((1,1)))
    p_loss.backward()
    paddle_optim.step()
    # test
    p_out.append(paddle_out.numpy())
    paddle_loss.append(p_loss.numpy())
    labels.append(label)

# paddle
reprod_logger = ReprodLogger()
reprod_logger.add("loss", np.array(paddle_loss).reshape(-1))
reprod_logger.save("Step4/bp_align_paddle.npy")
# diff
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("Step4/bp_align_torch.npy")
paddle_info = diff_helper.load_info("Step4/bp_align_paddle.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="Diff/bp_align_diff.log")

In [ ]:
# step 5
# paddle
paddle_model.load_dict(paddle_para)
paddle_model.eval()
paddle_optim = paddle.optimizer.Adam(
            parameters=paddle_model.parameters(),
            learning_rate=0.001)
p_out = []
paddle_loss = []
for epoch in range(2):
    for data in pipline_data:
        data, label = data[:-1], data[-1]
        # paddle
        paddle_optim.clear_grad()
        paddle_out = paddle_model.forward(data)
        p_loss = paddle.nn.functional.binary_cross_entropy(paddle_out, paddle.to_tensor(float(label)).reshape((1,1)))
        p_loss.backward()
        paddle_optim.step()
        # test
        p_out.append(paddle_out.numpy())
        paddle_loss.append(p_loss.numpy())
    # print(torch_loss)
    # print(paddle_loss)
# paddle
reprod_logger = ReprodLogger()
reprod_logger.add("train", np.array(paddle_loss))
reprod_logger.save("Step5/train_paddle1.npy")
# step 5
diff_helper = ReprodDiffHelper()
torch_info = diff_helper.load_info("Step5/train_torch1.npy")
paddle_info = diff_helper.load_info("Step5/train_paddle1.npy")

diff_helper.compare_info(torch_info, paddle_info)

diff_helper.report(path="Diff/loss_diff.log")

## step3

In [104]:
from sklearn.metrics import roc_auc_score
label = np.array([0, 0, 1, 1])
pred = np.array([0.1, 0.4, 0.35, 0.8])
# torch
print(roc_auc_score(label, pred))
# paddle
m = paddle.metric.Auc()

class1_preds = pred.reshape(-1, 1)
class0_preds = 1 - class1_preds

preds = np.concatenate((class0_preds, class1_preds), axis=1)

m.update(preds=preds, labels=label)
print(m.accumulate())

0.75
0.75
